In [3]:
import unittest
import torch
from collections import deque


# Класс Node (Узел) для скалярного автоматического дифференцирования

In [4]:

class Node:
    """
    Реализует базовые функции autograd для скалярных значений.
    Поддерживает: сложение (+), умножение (*) и функцию активации ReLU.
    """
    def __init__(self, data, _children=(), _op=''):
        # Скалярное значение данных
        self.data = data
        # Градиент (производная целевой функции по этому узлу)
        self.grad = 0.0
        # Функция для вычисления градиентов этого узла, вызванная из родительского узла
        self._backward = lambda: None
        # Множество дочерних узлов (входные данные для текущей операции)
        self._prev = set(_children)
        # Название операции, создавшей этот узел (для отладки/отображения)
        self._op = _op

    def __repr__(self):
        """Возвращает строковое представление узла."""
        # Используем "Node" в выводе, как в примере пользователя.
        return f"Node(data={self.data}, grad={self.grad})"

    # --- Операция сложения ---
    def __add__(self, other):
        """Сложение: self + other"""
        # Преобразование скаляра в Node для унификации
        other = other if isinstance(other, Node) else Node(other)
        
        # Создание выходного узла
        out = Node(self.data + other.data, (self, other), '+')

        # Функция обратного прохода для сложения (d(out)/da = 1, d(out)/db = 1)
        def _backward():
            # Принцип цепного правила: локальный градиент * градиент родителя
            # Применяем += для аккумуляции градиентов (если узел используется в нескольких местах)
            self.grad += out.grad * 1.0
            other.grad += out.grad * 1.0
        out._backward = _backward
        
        return out

    # Поддержка обратного сложения (other + self), например, 5 + Node(10)
    def __radd__(self, other):
        return self + other

    # --- Операция умножения ---
    def __mul__(self, other):
        """Умножение: self * other"""
        other = other if isinstance(other, Node) else Node(other)
        
        out = Node(self.data * other.data, (self, other), '*')

        # Функция обратного прохода для умножения (d(out)/da = b, d(out)/db = a)
        def _backward():
            # d(out)/d(self) = other.data
            self.grad += out.grad * other.data
            # d(out)/d(other) = self.data
            other.grad += out.grad * self.data
        out._backward = _backward
        
        return out

    # Поддержка обратного умножения (other * self), например, 5 * Node(10)
    def __rmul__(self, other):
        return self * other

    # --- Функция активации ReLU ---
    def relu(self):
        """
        Rectified Linear Unit (ReLU)
        out = max(0, data)
        """
        out_data = self.data if self.data > 0 else 0.0
        out = Node(out_data, (self,), 'relu')

        # Функция обратного прохода для ReLU
        def _backward():
            # d(out)/d(self) = 1, если self.data > 0, иначе 0
            self.grad += out.grad * (1.0 if self.data > 0 else 0.0)
        out._backward = _backward
        
        return out

    # --- Обратное распространение ошибки ---
    def backward(self):
        """
        Выполняет обратное распространение градиентов по вычислительному графу.
        Используется топологическая сортировка для обеспечения корректного порядка вычислений.
        """
        # 1. Топологическая сортировка узлов графа (обход в глубину)
        topo = []
        visited = set()
        
        def build_topo(v):
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    build_topo(child)
                topo.append(v)
        
        build_topo(self)
        
        # 2. Инициализация градиента корневого узла (целевой функции)
        # d(L)/d(L) = 1
        self.grad = 1.0
        
        # 3. Обратный проход по графу (в обратном топологическом порядке)
        for node in reversed(topo):
            node._backward()
            



    
    


# МОДУЛЬНЫЕ ТЕСТЫ С ИСПОЛЬЗОВАНИЕМ UNITTEST


In [5]:
class TestAutograd(unittest.TestCase):
    
    def test_example_from_prompt(self):
        """Тестирование примера из задания: a + b * c | relu"""
        a = Node(2.0)
        b = Node(-3.0)
        c = Node(10.0)
        
        d = a + b * c  # d = 2 + (-3 * 10) = -28
        e = d.relu()   # e = max(0, -28) = 0
        
        # Проверка прямого прохода
        self.assertAlmostEqual(e.data, 0.0)
        
        # Обратный проход
        e.backward()
        
        # dL/dd = 0, т.к. d < 0 после ReLU.
        # Все градиенты входных узлов должны быть 0.
        
        self.assertAlmostEqual(a.grad, 0.0, msg="Градиент 'a' должен быть 0 (d < 0 после ReLU)")
        self.assertAlmostEqual(b.grad, 0.0, msg="Градиент 'b' должен быть 0 (d < 0 после ReLU)")
        self.assertAlmostEqual(c.grad, 0.0, msg="Градиент 'c' должен быть 0 (d < 0 после ReLU)")
        self.assertAlmostEqual(d.grad, 0.0, msg="Градиент 'd' должен быть 0 (градиент ReLU при d < 0)")
        self.assertAlmostEqual(e.grad, 1.0, msg="Градиент 'e' (корневой узел) должен быть 1")
        
        
    def test_chain_rule_and_mixed_ops(self):
        """Тестирование сложной цепи операций (добавление, умножение)"""
        x1 = Node(2.0)
        x2 = Node(0.0)
        w1 = Node(-3.0)
        w2 = Node(1.0)
        b = Node(6.8813735870195432) # Имитация смещения
        
        # Вычисление: L = (x1 * w1) + (x2 * w2) + b
        x1w1 = x1 * w1
        x2w2 = x2 * w2
        x1w1x2w2 = x1w1 + x2w2
        n = x1w1x2w2 + b # n = -6.0 + 0.0 + 6.88... = 0.88...
        
        # ReLU для проверки градиента > 0
        o = n.relu() # o = 0.88...
        
        self.assertAlmostEqual(o.data, 0.8813735870195432)
        
        o.backward()
        
        # Проверка градиентов (o.grad=1, n.grad=1, т.к. n > 0):
        # dL/db = 1.0
        # dL/dw1 = dL/dn * dn/dx1w1 * dx1w1/dw1 = 1.0 * 1.0 * x1.data = 2.0
        # dL/dx1 = dL/dn * dn/dx1w1 * dx1w1/dx1 = 1.0 * 1.0 * w1.data = -3.0
        # dL/dw2 = dL/dn * dn/dx2w2 * dx2w2/dw2 = 1.0 * 1.0 * x2.data = 0.0
        # dL/dx2 = dL/dn * dn/dx2w2 * dx2w2/dx2 = 1.0 * 1.0 * w2.data = 1.0 (НЕ 0.0!)
        
        self.assertAlmostEqual(b.grad, 1.0)
        self.assertAlmostEqual(w1.grad, 2.0)
        self.assertAlmostEqual(x1.grad, -3.0)
        self.assertAlmostEqual(w2.grad, 0.0) 
        self.assertAlmostEqual(x2.grad, 1.0, msg="Градиент x2 должен быть 1.0 (w2.data), а не 0.0")


    def test_relu_negative_input(self):
        """Тестирование ReLU для отрицательного входного значения"""
        x = Node(-5.0)
        y = Node(10.0)
        
        # Результат = max(0, -5) * 10 = 0
        a = x.relu()
        b = a * y
        
        self.assertAlmostEqual(a.data, 0.0)
        self.assertAlmostEqual(b.data, 0.0)
        
        b.backward()
        
        # dL/dx = 0, т.к. x < 0.
        self.assertAlmostEqual(y.grad, 0.0) # dL/dy = a.data * dL/db = 0 * 1 = 0
        self.assertAlmostEqual(a.grad, 10.0)
        self.assertAlmostEqual(x.grad, 0.0)


    def test_pytorch_validation(self):
        """Валидация градиентов с помощью PyTorch (для ground truth)"""
        # Инициализация для Node
        a = Node(3.0)
        b = Node(4.0)
        c = Node(-2.0)
        
        # Инициализация для PyTorch
        # requires_grad=True для отслеживания операций
        ta = torch.tensor([3.0], requires_grad=True)
        tb = torch.tensor([4.0], requires_grad=True)
        tc = torch.tensor([-2.0], requires_grad=True)

        # Вычисление в Node: L = ReLU((a * b) + c)
        ab = a * b
        abc = ab + c
        L = abc.relu()
        L.backward()

        # Вычисление в PyTorch: L_torch = ReLU((ta * tb) + tc)
        tabc = (ta * tb) + tc
        L_torch = torch.relu(tabc)
        L_torch.backward()

        # Проверка градиентов
        self.assertAlmostEqual(L.data, L_torch.item()) # L = 10.0

        # Сравнение градиентов:
        # dL/da = 4.0
        # dL/db = 3.0
        # dL/dc = 1.0
        self.assertAlmostEqual(a.grad, ta.grad.item(), places=5, msg="Градиент 'a' не совпадает")
        self.assertAlmostEqual(b.grad, tb.grad.item(), places=5, msg="Градиент 'b' не совпадает")
        self.assertAlmostEqual(c.grad, tc.grad.item(), places=5, msg="Градиент 'c' не совпадает")


# ПРИМЕР ИСПОЛЬЗОВАНИЯ ИЗ ЗАДАНИЯ



In [6]:
if __name__ == '__main__':
    print("--- Запуск Примера Пользователя ---")
    
    # Сброс градиентов для чистоты (при запуске как main)
    def reset_grads(*nodes):
        for node in nodes:
            node.grad = 0.0
            
    a = Node(2)
    b = Node(-3)
    c = Node(10)
    
    # Сохраняем исходные данные для печати, как в примере
    a_data = a.data
    b_data = b.data
    c_data = c.data
    
    d = a + b * c # d = 2 + (-3 * 10) = -28
    e = d.relu()  # e = max(0, -28) = 0
    
    e.backward()
    
    print(f"Input: a={a_data}, b={b_data}, c={c_data}")
    print(f"Operation: d = a + b * c, e = d.relu()")
    
    print("\nOutput (после e.backward()):")
    
    # Выводим фактические корректные результаты:
    # Градиенты a, b, c, d равны 0.0, т.к. градиент ReLU для отрицательного входа равен 0.
    print(a) 
    print(b) 
    print(c) 
    print(d) 
    print(e) # Корневой узел, grad = 1.0
    
    # ---------------------------------------------
    print("\n--- Запуск Модульных Тестов Unittest ---")
    
    # Используем TestLoader.loadTestsFromTestCase() для избежания DeprecationWarning
    loader = unittest.TestLoader()
    suite = loader.loadTestsFromTestCase(TestAutograd)
    
    # Запуск тестов
    runner = unittest.TextTestRunner(verbosity=2)
    runner.run(suite)
    
    # ---------------------------------------------
    print("\n--- Пример с положительным ReLU ---")
    reset_grads(a, b, c)
    x = Node(5.0)
    y = Node(-2.0)
    z = x + y * Node(1.0) # z = 5 + (-2) * 1 = 3
    L = z.relu() # L = 3
    L.backward()
    
    print(f"L.data: {L.data}, L.grad: {L.grad}")
    print(f"x.grad: {x.grad}") # Должно быть 1.0
    print(f"y.grad: {y.grad}") # Должно быть 1.0

test_chain_rule_and_mixed_ops (__main__.TestAutograd.test_chain_rule_and_mixed_ops)
Тестирование сложной цепи операций (добавление, умножение) ... ok
test_example_from_prompt (__main__.TestAutograd.test_example_from_prompt)
Тестирование примера из задания: a + b * c | relu ... ok
test_pytorch_validation (__main__.TestAutograd.test_pytorch_validation)
Валидация градиентов с помощью PyTorch (для ground truth) ... ok
test_relu_negative_input (__main__.TestAutograd.test_relu_negative_input)
Тестирование ReLU для отрицательного входного значения ... ok

----------------------------------------------------------------------
Ran 4 tests in 0.005s

OK


--- Запуск Примера Пользователя ---
Input: a=2, b=-3, c=10
Operation: d = a + b * c, e = d.relu()

Output (после e.backward()):
Node(data=2, grad=0.0)
Node(data=-3, grad=0.0)
Node(data=10, grad=0.0)
Node(data=-28, grad=0.0)
Node(data=0.0, grad=1.0)

--- Запуск Модульных Тестов Unittest ---

--- Пример с положительным ReLU ---
L.data: 3.0, L.grad: 1.0
x.grad: 1.0
y.grad: 1.0
